In [ ]:
import numpy as np
import pandas as pd
import os
import tempfile

def calculate_performance_metrics(Contacts_Measured, Contacts, Measured, Random_Points, input_files):
    """
    Calculate performance metrics based on input DataFrames
    
    Parameters:
    - Contacts_Measured: DataFrame with contact and measurement information
    - Contacts: DataFrame with ground truth contacts
    - Measured: DataFrame with extracted measurements
    - Random_Points: DataFrame with random points
    - input_files: Dictionary of input file paths
    
    Returns:
    - Dictionary of performance metrics
    """
    # Calculate True Positives (TP)
    Contacts_Measured_Az_Match = Contacts_Measured[Contacts_Measured['Az_Match'] == 1]
    result_Contacts_Measured = Contacts_Measured_Az_Match.drop_duplicates(subset=['IN_FID'])
    arr_Contacts_Measured = result_Contacts_Measured[['IN_FID']].to_numpy()
    TP = len(arr_Contacts_Measured)

    # Calculate part of False Negatives (FN_1)
    Contacts_Measured_Az_Match_1 = Contacts_Measured[Contacts_Measured['Az_Match'] == 0]
    result_Contacts_Measured_1 = Contacts_Measured_Az_Match_1.drop_duplicates(subset=['IN_FID'])
    arr_Contacts_Measured_1 = result_Contacts_Measured_1[['IN_FID']].to_numpy()
    
    # Process Contacts and Measured DataFrames
    result_Contacts = Contacts.drop_duplicates(subset=['IN_FID'])
    arr_Contacts = result_Contacts[['IN_FID']].to_numpy()
    
    result_Measured = Measured.drop_duplicates(subset=['IN_FID'])
    arr_Measured = result_Measured[['IN_FID']].to_numpy()

    # Calculate intersections
    intersect_contacts_measured = np.intersect1d(arr_Contacts, arr_Measured)
    
    # Calculate False Negatives
    FN_2 = len(arr_Measured) - len(intersect_contacts_measured)
    
    # Calculate False Positives
    dif_contacts_measured = np.setdiff1d(arr_Contacts, arr_Measured)
    dif_measured_contacts = np.setdiff1d(arr_Measured, arr_Contacts)
    
    # Additional calculations for FN_1
    intersect = np.intersect1d(arr_Contacts_Measured, arr_Contacts_Measured_1)
    FN_1_arr = np.setdiff1d(arr_Contacts_Measured_1, intersect)
    FN_1 = len(arr_Contacts_Measured_1) - len(intersect)

    # Calculate True Negatives (TN)
    Random_Points_TN = Random_Points[Random_Points['NEAR_ANGLE'] == 0]
    random_points_arr = Random_Points_TN[['OBJECTID']].to_numpy()
    
    TN_1 = np.intersect1d(random_points_arr, arr_Measured)
    TN_2 = np.intersect1d(random_points_arr, arr_Contacts)
    TN_1_1 = np.intersect1d(TN_1, TN_2)
    
    TN = len(random_points_arr) - (len(TN_1) + len(TN_2)) + len(TN_1_1)

    # Finalize False Negatives and False Positives
    FN = len(dif_contacts_measured) + len(FN_1_arr)
    FP = len(dif_measured_contacts)

    # Calculate Performance Metrics
    total_samples = TP + TN + FP + FN
    
    # Combine metrics with input file information
    metrics = {
        # Input file identifiers
        'Contacts_Measured_File': os.path.basename(input_files['contacts_measured']),
        'Contacts_File': os.path.basename(input_files['contacts']),
        'Measured_File': os.path.basename(input_files['measured']),
        'Random_Points_File': os.path.basename(input_files['random_points']),
        
        # Performance metrics
        'TP': TP,
        'TN': TN,
        'FP': FP,
        'FN': FN,
        'Total_Samples': total_samples,
        'Accuracy': (TP + TN) / total_samples,
        'Precision': TP / (TP + FP) if (TP + FP) > 0 else 0,
        'Recall': TP / (TP + FN) if (TP + FN) > 0 else 0,
        'F1_Score': TP / (TP + ((FN + FP) / 2)) if (TP + ((FN + FP) / 2)) > 0 else 0,
        'FPR': FP / (FP + TN) if (FP + TN) > 0 else 0,
        'Specificity': TN / (TN + FP) if (TN + FP) > 0 else 0
    }
    
    return metrics

def save_metrics_to_csv(metrics, output_file='performance_metrics.csv'):
    """
    Save performance metrics to a CSV file with robust error handling
    
    Parameters:
    - metrics: Dictionary of performance metrics
    - output_file: Path to the output CSV file
    """
    try:
        # Try to save to the specified location
        # Ensure the directory exists
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        
        # Check if file exists
        file_exists = os.path.isfile(output_file)
        
        # Convert metrics to DataFrame
        metrics_df = pd.DataFrame([metrics])
        
        # Append to existing file or create new
        if file_exists:
            metrics_df.to_csv(output_file, mode='a', header=False, index=False)
            print(f"Metrics appended to {output_file}")
        else:
            metrics_df.to_csv(output_file, index=False)
            print(f"Metrics saved to new file {output_file}")
    
    except PermissionError:
        # If permission is denied, use a temporary file
        temp_file = os.path.join(tempfile.gettempdir(), 'performance_metrics.csv')
        
        # Check if file exists
        file_exists = os.path.isfile(temp_file)
        
        # Convert metrics to DataFrame
        metrics_df = pd.DataFrame([metrics])
        
        # Append to existing file or create new
        if file_exists:
            metrics_df.to_csv(temp_file, mode='a', header=False, index=False)
            print(f"Permission denied. Metrics appended to temporary file: {temp_file}")
        else:
            metrics_df.to_csv(temp_file, index=False)
            print(f"Permission denied. Metrics saved to temporary file: {temp_file}")
        
        # Suggest alternative save location
        print("Unable to save to specified location due to permission issues.")
        print(f"Temporary file saved at: {temp_file}")
        print("Please copy this file to your desired location manually.")

# Example usage (you would replace these with your actual DataFrames)
def main():
    # Manually input or load your DataFrames
    print("Please input the file paths for your DataFrames:")
    contacts_measured_path = input("Path to Contacts_Measured CSV: ")
    contacts_path = input("Path to Contacts CSV: ")
    measured_path = input("Path to Measured CSV: ")
    random_points_path = input("Path to Random_Points CSV: ")
    
    # Create input files dictionary
    input_files = {
        'contacts_measured': contacts_measured_path,
        'contacts': contacts_path,
        'measured': measured_path,
        'random_points': random_points_path
    }
    
    # Prompt for output file path with a default
    print("\nWhere would you like to save the performance metrics?")
    print("1. Use default temporary location")
    print("2. Specify a custom path")
    choice = input("Enter 1 or 2: ").strip()
    
    if choice == '2':
        output_file = input("Enter full path to save performance metrics CSV (including filename): ")
    else:
        # Use a default name in the temporary directory
        output_file = os.path.join(tempfile.gettempdir(), 'performance_metrics.csv')
    
    # Load DataFrames
    Contacts_Measured = pd.read_csv(contacts_measured_path)
    Contacts = pd.read_csv(contacts_path)
    Measured = pd.read_csv(measured_path)
    Random_Points = pd.read_csv(random_points_path)
    
    # Calculate metrics
    metrics = calculate_performance_metrics(Contacts_Measured, Contacts, Measured, Random_Points, input_files)
    
    # Print metrics to console
    for key, value in metrics.items():
        print(f"{key}: {value}")
    
    # Save metrics to CSV
    save_metrics_to_csv(metrics, output_file)

if __name__ == "__main__":
    main()

Please input the file paths for your DataFrames:


Path to Contacts_Measured CSV:  
Path to Contacts CSV:  
Path to Measured CSV:  
Path to Random_Points CSV:  



Where would you like to save the performance metrics?
1. Use default temporary location
2. Specify a custom path
